In [1]:
import numpy as np
from scipy.spatial import distance
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  #3D plot

经验势的表示函数

In [2]:
def emperical_potential(r1,r2,a=5,b=10):
    d = distance.euclidean(r1,r2)
    return a / (d ** 12) - b / (d ** 6)

In [3]:
def cutoff_func(r_ij,r_c = 5.0):  #f_c(R_ij)
    if r_ij > r_c:
        return 0
    else:
        return 0.5*(np.cos(np.pi * r_ij / r_c) + 1)

原子类,包含原子坐标

In [4]:
class Atom():
    def __init__(self,coords):
        self.coords = coords
        self.individual_potential = 0
        self.g1_value = 0  #镜像对称函数值
        self.g2_value = 0  #镜像对称函数值
        self.g3_value = 0  #镜像对称函数值
    def individual_potential_cal(self,neighbors):
        for neighbor in neighbors:  #neighbors为相对距离列表
            self.individual_potential += 0.5 * emperical_potential(0,neighbor)
    def g1_cal(self,neighbors):
        for neighbor in neighbors:
            self.g1_value += cutoff_func(neighbor)
    def g2_cal(self,neighbors,eta = 0.1,r_s = 0.1):
        for neighbor in neighbors:
            self.g2_value += np.exp(-eta * (neighbor-r_s)**2)*cutoff_func(neighbor)
    def g3_cal(self,neighbors,kappa = 0.5):
        for neighbor in neighbors:
            self.g3_value += np.cos(kappa*neighbor)*cutoff_func(neighbor)

原子点集类,含维数、总势能、原子个数、坐标范围等参数,并有随机生成、输入列表生成、势能计算等方法

In [5]:
class AtomCollection():
    def __init__(self):
        """
        Args:
        coords_range: suppose (x, ...) is the coordinate of an atom while range is (a,b), then a<x<b, ...
        """
        self.atoms = []
        self.num = 0
        self.dimension = 0
        self.potential = 0
        self.coords_range=(-5,5)
        
    def random_generate(self,size,dimension=2):
        """generate a collection of atoms by using random numbers
        
        Args:
            size: the number of atoms in self.atoms
            dimension: 2 or 3
        """
        flag = True
        self.dimension = dimension
        temp_coords = np.random.uniform(self.coords_range[0],self.coords_range[1],
                                        dimension).tolist()  #1*dimension shape-like list
        self.num += 1
        self.atoms.append(Atom(temp_coords))
        while self.num < size:
            temp_coords = np.random.uniform(self.coords_range[0],self.coords_range[1],
                                            dimension).tolist()
            for atom in self.atoms:
                if distance.euclidean(atom.coords,temp_coords) < 0.1:  #太近了先不考虑
                    flag = False
                    break
            if flag is True:
                self.num +=1
                self.atoms.append(Atom(temp_coords))
                    
    def list_generate(self,coords_list):
        """generate a collection of atoms by using a given list
        
        Args:
            coords_list: shape like [[1,1,...],[2,2,...],...]
        """
        if coords_list == []:
            return
        for coords in coords_list:
            self.atoms.append(Atom(coords))
        self.num = len(coords_list)
        self.dimension = len(coords_list[0])
    
    def distance_cal(self,center):  #计算体系中其他所有原子和中心原子的相对距离
        dis = []
        for atom in self.atoms:
            if atom is not center:
                dis.append(distance.euclidean(atom.coords,center.coords))
        return dis
    
    def potential_cal(self):
        """calculate the total potential of the atom collection"""        
        for atom in self.atoms: #将每个原子的势能累加得到体系总势能
            self.potential += atom.individual_potential
        
    def plot_atoms(self):
        """plot all the atoms on one figure"""
        fig = plt.figure()
        
        if self.dimension == 2:
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=self.coords_range, ylim=self.coords_range)
            for atom in self.atoms:
                ax.scatter(atom.coords[0],atom.coords[1], color = 'black')
            
        if self.dimension == 3:
            ax = Axes3D(fig)
            for atom in self.atoms:
                ax.scatter(atom.coords[0], atom.coords[1],atom.coords[2])
            ax.set_zlabel('Z', fontdict={'size': 15, 'color': 'black'})
            ax.set_ylabel('Y', fontdict={'size': 15, 'color': 'black'})
            ax.set_xlabel('X', fontdict={'size': 15, 'color': 'black'})
    
    def clear_collection(self):
        """erase all the coordinates"""
        self.atoms = []
        self.num = 0
        self.dimension = 0
        self.potential = 0

神经网络训练

In [6]:
import tensorflow as tf

In [7]:
def data_reshape(atom_collection_set):
    """reshape the collections, with output like [[g11,g12,g13],[g21,g22,g23],...] & [E1,E2,...]"""
    g_factors = []
    potentials = []
    for collection in atom_collection_set:
        for atom in collection.atoms:
            #g_factors.append([atom.g1_value,atom.g2_value])
            g_factors.append([atom.g1_value,atom.g2_value,atom.g3_value])
            potentials.append(atom.individual_potential)
            #print(atom.individual_potential)
    return np.array(g_factors),np.array(potentials)

In [8]:
def get_batch(params,energies,batch_size):
    index = np.random.randint(0, len(energies), batch_size)
    return params[index],energies[index]

In [9]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100)
        self.dense2 = tf.keras.layers.Dense(units=50,activation = tf.nn.tanh)
        self.dense3 = tf.keras.layers.Dense(units=1)

    def call(self, inputs):         
        x = self.flatten(inputs)    # [batch_size, atom_size*number_of_factors]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 1]
        x = self.dense3(x)          # [batch_size, 1]
        output = x
        return output

In [10]:
#超参数hyperparameters
num_epoch = 1.0
batch_size = 1
rate = 0.00000001
atom_size = 50
configurations = 400
dim = 3
test_size = 20

In [11]:
#建训练集
config = 0
train_collections = []
r_ij = []
while config < configurations:
    temp = AtomCollection()
    temp.random_generate(size = atom_size,dimension = dim)
    for atom in temp.atoms:
        r_ij = temp.distance_cal(atom)
        atom.individual_potential_cal(r_ij)
        atom.g1_cal(r_ij)
        atom.g2_cal(r_ij)
        atom.g3_cal(r_ij)
        r_ij = []
    train_collections.append(temp)
    config += 1

In [12]:
g_factors,potentials = data_reshape(train_collections)
#g_factors[:15]
#potentials[:15]

In [13]:
g_factors.shape

(20000, 3)

In [14]:
potentials.shape

(20000,)

In [15]:
#训练
model = MLP()
optimizer = tf.keras.optimizers.Adam(learning_rate=rate)
num_batches = int(len(potentials)//batch_size*num_epoch)

for batch_index in range(num_batches):
    X, y = get_batch(g_factors,potentials,batch_size)  #X: (batch_size,num_of_factors), y: (batch_size,)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        y_temp = tf.convert_to_tensor(y.reshape(-1,1),dtype = 'float32')
        #print(y_temp)
        #print(y_pred)
        #print(tf.square(y_pred - y_temp))
        loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y_temp))
        print("batch {}: loss {}".format(batch_index,loss.numpy()))
        if batch_index > 0.8 * num_batches and loss.numpy() < 2e-8:
            break
    grad = tape.gradient(loss,model.variables)  #model.variables直接调用模型变量
    optimizer.apply_gradients(grads_and_vars=zip(grad,model.variables))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

batch 0: loss 0.006181038916110992
batch 1: loss 0.0006813546642661095
batch 2: loss 3.8823397829901296e-08
batch 3: loss 0.0017447430873289704
batch 4: loss 0.0022290886845439672
batch 5: loss 0.004996960051357746
batch 6: loss 0.08475212752819061
batch 7: loss 0.00020044369739480317
batch 8: loss 0.0037583138328045607
batch 9: loss 0.006610841490328312
batch 10: loss 0.07450661808252335
batch 11: loss 0.006404442712664604
batch 12: loss 0.003457501530647278
batch 13: loss 0.0016690987395122647
batch 1

batch 203: loss 0.0015378074022009969
batch 204: loss 0.5662599802017212
batch 205: loss 0.002005231799557805
batch 206: loss 0.006264620926231146
batch 207: loss 0.0031477296724915504
batch 208: loss 0.0012708243448287249
batch 209: loss 0.0016770210349932313
batch 210: loss 0.015261025168001652
batch 211: loss 0.022606171667575836
batch 212: loss 2.5917830498656258e-05
batch 213: loss 0.002287960611283779
batch 214: loss 0.010419865138828754
batch 215: loss 0.0003351795603521168
batch 216: loss 4.102760087221213e-08
batch 217: loss 0.01416847761720419
batch 218: loss 0.009090671315789223
batch 219: loss 6.756077652880776e-08
batch 220: loss 0.0011896578362211585
batch 221: loss 0.00249344645999372
batch 222: loss 0.006395295262336731
batch 223: loss 1.4516291230393108e-05
batch 224: loss 0.0005862188991159201
batch 225: loss 0.00022363557945936918
batch 226: loss 0.005771607626229525
batch 227: loss 0.0015990820247679949
batch 228: loss 0.0019229765748605132
batch 229: loss 0.0017735

batch 425: loss 0.0012372343335300684
batch 426: loss 0.004638894461095333
batch 427: loss 0.010769002139568329
batch 428: loss 5.6682940339669585e-05
batch 429: loss 0.00042918117833323777
batch 430: loss 0.009780431166291237
batch 431: loss 1.902834512179652e-08
batch 432: loss 2.629972457885742
batch 433: loss 2.1700005531311035
batch 434: loss 0.0035847951658070087
batch 435: loss 3.21957741107326e-05
batch 436: loss 4.198761871521128e-06
batch 437: loss 5.880382013856433e-05
batch 438: loss 0.0035847951658070087
batch 439: loss 0.002013379242271185
batch 440: loss 8.658915612613782e-05
batch 441: loss 0.0056554642505943775
batch 442: loss 0.0012916622217744589
batch 443: loss 5.095879984651219e-08
batch 444: loss 1.5225093364715576
batch 445: loss 0.004053574986755848
batch 446: loss 0.002017709892243147
batch 447: loss 0.002928361762315035
batch 448: loss 0.3102013170719147
batch 449: loss 0.0010507594561204314
batch 450: loss 0.0024620937183499336
batch 451: loss 0.0127937868237

batch 650: loss 4.508878191700205e-06
batch 651: loss 5.6288963605766185e-06
batch 652: loss 3.6570014572134824e-07
batch 653: loss 0.0006482023163698614
batch 654: loss 0.00015230070857796818
batch 655: loss 1.6765450538969162e-07
batch 656: loss 0.0006221742951311171
batch 657: loss 0.0076375394128263
batch 658: loss 2.7933989343864596e-08
batch 659: loss 0.0017934846691787243
batch 660: loss 0.16123215854167938
batch 661: loss 0.0029614658560603857
batch 662: loss 0.001424985472112894
batch 663: loss 0.0002149069623555988
batch 664: loss 0.006707489024847746
batch 665: loss 0.0008151991642080247
batch 666: loss 0.0018851133063435555
batch 667: loss 2.519627332687378
batch 668: loss 3.144957627654321e-09
batch 669: loss 0.0030751279555261135
batch 670: loss 0.007806512527167797
batch 671: loss 0.005354264751076698
batch 672: loss 3.967527663917281e-05
batch 673: loss 0.0003829730849247426
batch 674: loss 0.0019603969994932413
batch 675: loss 0.001822684658691287
batch 676: loss 0.001

batch 876: loss 0.011086002923548222
batch 877: loss 1.5932151598008204e-07
batch 878: loss 0.0005924970027990639
batch 879: loss 1.6679528869190108e-08
batch 880: loss 0.0020788495894521475
batch 881: loss 1.3159403522422508e-07
batch 882: loss 0.00828839186578989
batch 883: loss 0.0010289768688380718
batch 884: loss 1.6853065490722656
batch 885: loss 0.0016498679760843515
batch 886: loss 0.003996727056801319
batch 887: loss 0.0006353813223540783
batch 888: loss 0.0003037492570001632
batch 889: loss 0.00326709495857358
batch 890: loss 0.0014980117557570338
batch 891: loss 1.576420061155659e-08
batch 892: loss 0.0019817790016531944
batch 893: loss 0.0008781530195847154
batch 894: loss 0.0073539502918720245
batch 895: loss 0.003925819415599108
batch 896: loss 0.0025996959302574396
batch 897: loss 4.433472327036725e-07
batch 898: loss 0.0022911743726581335
batch 899: loss 0.0028460260946303606
batch 900: loss 0.001652430510148406
batch 901: loss 0.00011138794798171148
batch 902: loss 0.0

...

batch 14962: loss 0.004607720300555229
batch 14963: loss 0.007015961222350597
batch 14964: loss 1.2156792763562407e-05
batch 14965: loss 0.0016734175151214004
batch 14966: loss 0.0003823409497272223
batch 14967: loss 0.006781612988561392
batch 14968: loss 0.0003700318338815123
batch 14969: loss 0.002376838820055127
batch 14970: loss 0.01557963527739048
batch 14971: loss 0.12215565890073776
batch 14972: loss 0.0024585267528891563
batch 14973: loss 0.12386395037174225
batch 14974: loss 0.03305050730705261
batch 14975: loss 0.0013882758794352412
batch 14976: loss 0.002650914015248418
batch 14977: loss 0.0003252899332437664
batch 14978: loss 0.0035356292501091957
batch 14979: loss 0.06393276154994965
batch 14980: loss 0.0005276435404084623
batch 14981: loss 8.825417171465233e-05
batch 14982: loss 0.00021838747488800436
batch 14983: loss 0.009623021818697453
batch 14984: loss 8.539499685866758e-05
batch 14985: loss 8.567766940359434e-08
batch 14986: loss 4.631553383660503e-05
batch 14987: l

batch 15183: loss 0.0004167143488302827
batch 15184: loss 0.0009196832543238997
batch 15185: loss 0.0028270543552935123
batch 15186: loss 0.0003932862600777298
batch 15187: loss 0.00020305145881138742
batch 15188: loss 3.0376543236343423e-06
batch 15189: loss 0.006067159119993448
batch 15190: loss 0.0005697289598174393
batch 15191: loss 0.002075875410810113
batch 15192: loss 0.005248077213764191
batch 15193: loss 0.0013059022603556514
batch 15194: loss 2.93581178993918e-05
batch 15195: loss 0.003966478165239096
batch 15196: loss 0.005904987454414368
batch 15197: loss 0.00024527424830012023
batch 15198: loss 0.0015531813260167837
batch 15199: loss 0.0016351742669939995
batch 15200: loss 0.0016868396196514368
batch 15201: loss 0.0015414580702781677
batch 15202: loss 2.586580762908852e-07
batch 15203: loss 0.0005666845827363431
batch 15204: loss 9.36603257173374e-08
batch 15205: loss 0.0006313787307590246
batch 15206: loss 0.019811544567346573
batch 15207: loss 2.8036964067723602e-05
batc

batch 15402: loss 0.0007903135847300291
batch 15403: loss 0.0037520357873290777
batch 15404: loss 2.5106483008130454e-05
batch 15405: loss 0.000518239161465317
batch 15406: loss 3.0495058922497265e-08
batch 15407: loss 0.0005101576680317521
batch 15408: loss 0.008909628726541996
batch 15409: loss 0.020671801641583443
batch 15410: loss 0.0058676861226558685
batch 15411: loss 1426076.875
batch 15412: loss 0.002719619544222951
batch 15413: loss 0.0014915127540007234
batch 15414: loss 573.1826782226562
batch 15415: loss 8.08814988317863e-08
batch 15416: loss 0.010214105248451233
batch 15417: loss 0.0008779650670476258
batch 15418: loss 0.0012051734374836087
batch 15419: loss 0.002943202620372176
batch 15420: loss 6.600351412089367e-07
batch 15421: loss 5.640254130412359e-07
batch 15422: loss 0.023373344913125038
batch 15423: loss 0.0011979681439697742
batch 15424: loss 1.3342765114998656e-08
batch 15425: loss 0.0028990679420530796
batch 15426: loss 0.008535760454833508
batch 15427: loss 0.

batch 15615: loss 0.00029349065152928233
batch 15616: loss 6.385571032296866e-05
batch 15617: loss 1.4224529820694443e-07
batch 15618: loss 2.348462658119388e-05
batch 15619: loss 0.003788392525166273
batch 15620: loss 0.005298396106809378
batch 15621: loss 0.00011349641135893762
batch 15622: loss 0.003936802037060261
batch 15623: loss 0.0005233064293861389
batch 15624: loss 0.00475109601393342
batch 15625: loss 18120688.0
batch 15626: loss 0.00033360515953972936
batch 15627: loss 0.02276458591222763
batch 15628: loss 2.836150088114664e-05
batch 15629: loss 0.0040914397686719894
batch 15630: loss 4.945385080645792e-05
batch 15631: loss 0.00010456359450472519
batch 15632: loss 1.9494255099061775e-08
batch 15633: loss 0.0008081219275481999
batch 15634: loss 8.08325637535745e-08
batch 15635: loss 0.003048032522201538
batch 15636: loss 6.880436842493509e-08
batch 15637: loss 0.027573902159929276
batch 15638: loss 0.0005168315256014466
batch 15639: loss 0.0002168589417124167
batch 15640: lo

batch 15824: loss 0.001111188088543713
batch 15825: loss 0.001293563051149249
batch 15826: loss 9.36933997053302e-09
batch 15827: loss 0.0015609480906277895
batch 15828: loss 0.00033757713390514255
batch 15829: loss 0.0043669394217431545
batch 15830: loss 3.4142518416047096e-05
batch 15831: loss 0.0012684025568887591
batch 15832: loss 0.0010168134467676282
batch 15833: loss 0.0006102015031501651
batch 15834: loss 1.1108103308288264e-06
batch 15835: loss 0.0024360432289540768
batch 15836: loss 0.009065757505595684
batch 15837: loss 0.0010878872126340866
batch 15838: loss 2.7051806128497446e-08
batch 15839: loss 4.778319635079242e-06
batch 15840: loss 0.0023909788578748703
batch 15841: loss 0.0058676861226558685
batch 15842: loss 0.005106554366648197
batch 15843: loss 0.004864370916038752
batch 15844: loss 0.002743861172348261
batch 15845: loss 1.2864759035835505e-08
batch 15846: loss 0.001517434953711927
batch 15847: loss 7.390352863012595e-08
batch 15848: loss 6.534426211146638e-05
bat

In [16]:
#建测试集
test_collections = []
config = 0
while config < test_size:
    r_ij = []
    temp = AtomCollection()
    temp.random_generate(size = atom_size,dimension = dim)
    for atom in temp.atoms:
        r_ij = temp.distance_cal(atom)
        atom.individual_potential_cal(r_ij)
        atom.g1_cal(r_ij)
        atom.g2_cal(r_ij)
        atom.g3_cal(r_ij)
        #print(atom.individual_potential)
    temp.potential_cal()
    test_collections.append(temp)
    config += 1

In [17]:
#测试评估
energy_delta = []
for test_sample in test_collections:
    X_test,y_test = data_reshape([test_sample])
    #print(y_test)
    #print(X_test)
    y_pred = model.predict(X_test)
    #print(y_pred)
    total_potential = y_pred.sum()
    energy_delta.append(abs((total_potential - test_sample.potential) / test_sample.potential))

In [18]:
energy_delta

[1.0007354880768857,
 1.4028012120514466,
 1.0282154862365456,
 0.6719912404944236,
 0.22962637026467797,
 1.0000887676654209,
 0.10512649536525998,
 4.321613541787945,
 1.0280617130966068,
 0.48097167044025946,
 0.42302348348518054,
 0.5238950935371737,
 0.9741607107679988,
 1.0000308727251155,
 0.32276006085922687,
 0.9750992304576777,
 1.0054917909204117,
 0.16987898915500127,
 1.0751401712218271,
 1.0041368743170982]

In [19]:
energy_delta.sort()

In [20]:
print(energy_delta)

[0.10512649536525998, 0.16987898915500127, 0.22962637026467797, 0.32276006085922687, 0.42302348348518054, 0.48097167044025946, 0.5238950935371737, 0.6719912404944236, 0.9741607107679988, 0.9750992304576777, 1.0000308727251155, 1.0000887676654209, 1.0007354880768857, 1.0041368743170982, 1.0054917909204117, 1.0280617130966068, 1.0282154862365456, 1.0751401712218271, 1.4028012120514466, 4.321613541787945]
